In [56]:
import numpy as np
import pandas as pd
import torch
import sys

sys.path.append('/home/benoit.sautydechalon/deformetrica')

from deformetrica.core.estimator_tools.samplers.srw_mhwg_sampler import SrwMhwgSampler
from deformetrica.core.estimators.gradient_ascent import GradientAscent
from deformetrica.core.estimators.mcmc_saem import McmcSaem
# Estimators
from deformetrica.core.estimators.scipy_optimize import ScipyOptimize
from deformetrica.core.model_tools.manifolds.exponential_factory import ExponentialFactory
from deformetrica.core.model_tools.manifolds.generic_spatiotemporal_reference_frame import GenericSpatiotemporalReferenceFrame
from deformetrica.core.models.longitudinal_metric_learning import LongitudinalMetricLearning
from deformetrica.core.models.model_functions import create_regular_grid_of_points
from deformetrica.in_out.array_readers_and_writers import read_2D_array
from deformetrica.in_out.dataset_functions import read_and_create_scalar_dataset, read_and_create_image_dataset
from deformetrica.support.probability_distributions.multi_scalar_normal_distribution import MultiScalarNormalDistribution
from deformetrica.support.utilities.general_settings import Settings
from deformetrica import estimate_longitudinal_metric_model
from deformetrica.in_out.array_readers_and_writers import *
from deformetrica.launch.estimate_longitudinal_metric_model import instantiate_longitudinal_metric_model
import deformetrica as dfca

## 1. Loading the individual parameters and raw data

In [57]:
path = 'bivariate_study'

In [136]:
rer = np.load(path+'/output/LongitudinalMetricModel__EstimatedParameters__IndividualRandomEffectsSamples.npy', 
              allow_pickle=True)[()]
trajectories = np.load(path+'/output/LongitudinalMetricModel__EstimatedParameters__Trajectory.npy',
       allow_pickle=True)[()]
ids = pd.read_csv(path+'/output/LongitudinalMetricModel_subject_ids_unique.txt', header=None).values

In [102]:
# First we put them in a dataframe for visualization purpose

ip = pd.DataFrame(columns=['tau','xi','source'], index=[int(idx[0]) for idx in ids])
ip['tau'] = rer['onset_age'].mean(axis=0)
ip['xi'] = rer['log_acceleration'].mean(axis=0)
ip['source'] = rer['sources'].mean(axis=0)

ip

,tau,xi,source
4,67.606189,-0.073793,0.452786
6,74.419056,-1.080613,-0.718563
7,66.140717,-0.071428,0.466941
16,81.893425,-0.729838,-0.078377
29,60.146789,0.872884,1.235304
...,...,...,...
1402,63.308329,0.286339,1.066510
1409,58.746042,0.348766,-0.103641
1419,85.071384,0.532479,1.062330
1425,62.906911,-0.746155,1.226602


In [142]:
individual_RER

{'onset_age': array([ 67.617074,  74.392572,  66.160545,  81.780836,  60.152984,
         81.630142,  69.537493,  82.317976,  84.654515,  78.398012,
         67.493437,  71.747115,  94.166713,  73.788951,  74.405582,
         70.921932,  61.952544,  77.308163,  69.793913,  82.233835,
         71.678106,  78.976767,  91.041312,  65.994319,  73.603783,
         65.767583,  80.327013,  68.220044,  80.109772,  75.817101,
         82.865414,  73.9837  ,  88.873142,  80.306068,  71.581463,
         81.12727 ,  64.304164,  57.84635 ,  83.795545,  63.046897,
         80.42443 ,  81.892076,  92.325672,  68.297595,  93.253546,
         93.784744,  71.256428,  83.470368,  64.085835,  76.859919,
         82.041788,  68.447061,  86.904603,  85.452811,  98.630529,
         89.395849,  92.312248,  77.459924,  71.500301,  70.007213,
         80.848471,  72.347424,  65.033448,  68.289443,  69.094374,
         75.770251,  76.915017,  77.643739,  72.366674,  55.160414,
         69.477685,  75.840955,  74

In [141]:
rer['onset_age'][-1]

array([ 68.2894253 ,  77.39602432,  64.84217569,  81.84986926,
        59.85100135,  81.69871006,  66.61997056,  82.46504284,
        86.50773304,  77.75213428,  65.88123022,  71.40390489,
       105.7985633 ,  76.1008559 ,  71.63108475,  70.78437745,
        60.15236369,  78.05055162,  73.41193589,  84.23614392,
        72.1555887 ,  76.28425995,  95.17220286,  68.28821035,
        73.95767239,  66.26455752,  81.34150995,  68.29888123,
        78.43323727,  76.13712023,  81.86655203,  75.54729692,
        91.20431262,  79.66830148,  71.93782324,  82.02067993,
        64.07767396,  57.90739565,  84.01490634,  63.48200456,
        80.10972212,  82.3818369 , 100.1081882 ,  68.96481173,
        93.2580864 ,  87.96505222,  72.17509851,  86.15570063,
        64.20735609,  79.810043  ,  82.96477186,  66.0402774 ,
        90.60654194,  85.97320894,  97.75968722,  93.23080056,
        92.0566157 ,  77.64084731,  73.276617  ,  70.25694899,
        81.09884653,  68.98407863,  66.96315459,  68.23

In [103]:
# Then in the required format for the deformetrica model

rer['onset_age'] = rer['onset_age'].mean(axis=0)
rer['log_acceleration'] = rer['log_acceleration'].mean(axis=0)
rer['sources'] = np.array([[source] for source in rer['sources'].mean(axis=0)])
averaged_rer = rer
averaged_rer

{'onset_age': array([ 67.60618949,  74.41905628,  66.14071748,  81.89342454,
         60.14678875,  81.62823344,  69.53361246,  82.32514039,
         84.68083294,  78.43670691,  67.49330881,  71.71732925,
         94.25442516,  73.72854262,  74.37217167,  70.91582779,
         61.9286866 ,  77.31320713,  69.77981526,  82.21828163,
         71.68515321,  78.88131915,  91.07455372,  66.01306741,
         73.59114472,  65.76306417,  80.40158112,  68.22999881,
         80.07310386,  75.81677425,  82.83848927,  73.98223395,
         88.8403536 ,  80.23788302,  71.59234461,  81.15598374,
         64.33163164,  57.80121215,  83.80203098,  63.05123313,
         80.4345927 ,  81.8298373 ,  92.41945854,  68.27427679,
         93.25661806,  93.80526621,  71.29632975,  83.51184077,
         64.11425737,  76.95369232,  82.04814597,  68.42450281,
         86.92691963,  85.49510879,  98.67072755,  89.43516852,
         92.3232145 ,  77.45635281,  71.52718607,  70.01767773,
         80.86197474,  72.2

In [137]:
trajectories['p0'] = trajectories['p0'][-1]
trajectories['v0'] = trajectories['v0'][-1]
trajectories['metric_parameters'] = trajectories['metric_parameters'][-1]
trajectories['modulation_matrix'] = trajectories['modulation_matrix'][-1]

In [114]:
write_2D_list(np.array([trajectories['p0']]), path+'/output/', 'final_p0.txt')
write_2D_list(np.array([trajectories['v0']]), path+'/output/', 'final_v0.txt')
write_2D_list(np.array([trajectories['metric_parameters']]), path+'/output/','final_metric_parameters.txt')
write_2D_list(np.array([trajectories['modulation_matrix']]), path+'/output/','final_modulation_matrix.txt')

In [107]:
trajectories['metric_parameters']

array([ 3.75245994e-01, -3.51101832e-01,  9.41286476e-02,  1.06105814e-05,
       -4.50893812e-02,  6.67341407e-02,  1.55430256e-05,  5.58230306e-02,
        1.22403073e-01,  1.21890902e-05,  3.49062546e-02,  2.92835138e-01,
        1.26382870e-05, -7.80066573e-02,  1.52329913e-01,  2.06949427e-01,
        1.42772792e-01,  8.63727498e-02,  2.22762587e-01,  3.17667652e-01,
        8.82492032e-02,  1.07999836e-05, -3.23940116e-02,  4.21928835e-01,
        1.19083374e-05, -6.01795353e-03,  2.94281993e-01,  8.75561094e-01,
        4.02018210e-01,  1.72076055e-01,  7.63871009e-06, -4.46528162e-02,
        2.00355603e-01,  1.55855141e-06, -2.48942782e-01,  4.33179890e-01,
        1.18805512e-05,  3.34243545e-03,  3.06768324e-01,  1.18635808e-05,
       -2.47464310e-03,  2.65167837e-01,  6.31380944e-06, -1.55997018e-01,
        2.68579165e-01,  8.50717226e-06, -1.58360066e-01,  2.97832953e-01])

In [158]:
times = [time[0] for time in pd.read_csv(path+'/output/LongitudinalMetricModel_times.txt', header=None).values]
adas_memory = [score[0] for score in pd.read_csv(path+'/bivariate_data/Y.csv', header=None).values]
hippocampus = [score[1] for score in pd.read_csv(path+'/bivariate_data/Y.csv', header=None).values]
ids = [int(idx[0]) for idx in pd.read_csv(path+'/output/LongitudinalMetricModel_subject_ids.txt', header=None).values]

In [159]:
data = pd.DataFrame(index=[ids, times], columns=['adas_memory', 'hippocampus'])
data['adas_memory'] = adas_memory
data['hippocampus'] = hippocampus
data

adas_memory  hippocampus
4    67.5     0.407333     0.377389
     68.0     0.518444     0.375055
     68.5     0.466667     0.364087
     69.0     0.540667     0.382969
     70.5     0.511111     0.429288
...                ...          ...
1425 77.6     0.540667     0.242699
     78.6     0.607333     0.267690
1430 83.4     0.577778     0.745385
     83.9     0.659333     0.774981
     85.4     0.770444     0.804698

[942 rows x 2 columns]

In [168]:
reconstructed = pd.read_csv('simulated_study/output/LongitudinalMetricModel_reconstructed_values.txt', sep=' ', header=None)
reconstructed.columns = ['adas_memory', 'hippocampus']
reconstructed.index = data.index


FileNotFoundError: [Errno 2] No such file or directory: 'simulated_study/output/LongitudinalMetricModel_reconstructed_values.txt'

In [167]:
abs(reconstructed-data).describe()

,adas_memory,hippocampus
count,942.000000,942.000000
mean,0.049836,0.020974
std,0.041588,0.018383
min,0.000067,0.000003
25%,0.020666,0.007649
50%,0.039522,0.015946
75%,0.067722,0.028693
max,0.372524,0.130248


In [156]:
data = pd.DataFrame(index=[ids, times], columns=['logistic', 'sum_logistic'])
data['logistic'] = logistic
data['sum_logistic'] = sum_logistic
data

ValueError: all arrays must be same length

## 2. Evaluating the reconstruction error

In [122]:
args = {'verbosity':'INFO', 'output':'personalize',
        'model':path+'/model_after_fit.xml', 'dataset':path+'/data_set.xml', 'parameters':path+'/optimization_parameters_saem.xml'}


"""
Read xml files, set general settings, and call the adapted function.
"""

xml_parameters = dfca.io.XmlParameters()
xml_parameters.read_all_xmls(args['model'],
                             args['dataset'],
                             args['parameters'])

logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.ERROR)
logger.setLevel(logging.INFO)

/home/benoit.sautydechalon/deformetrica/deformetrica/in_out/xml_parameters.py:579: UserWarning: Unknown entry while parsing the optimization_parameters xml: step-shrink
  warnings.warn(msg)
/home/benoit.sautydechalon/deformetrica/deformetrica/in_out/xml_parameters.py:579: UserWarning: Unknown entry while parsing the optimization_parameters xml: step-expand
  warnings.warn(msg)
/home/benoit.sautydechalon/deformetrica/deformetrica/in_out/xml_parameters.py:579: UserWarning: Unknown entry while parsing the optimization_parameters xml: number-of-threads
  warnings.warn(msg)


In [123]:
dataset = read_and_create_scalar_dataset(xml_parameters)
model, individual_RER = instantiate_longitudinal_metric_model(xml_parameters, logger, dataset, observation_type='scalar')

INFO:__main__:Setting initial onset ages from bivariate_study/output/LongitudinalMetricModel_onset_ages.txt file
INFO:__main__:Setting initial log accelerations from bivariate_study/output/LongitudinalMetricModel_log_accelerations.txt file
INFO:__main__:Initializing exponential type to parametric
INFO:__main__:Loading metric parameters from file bivariate_study/output/final_metric_parameters.txt
INFO:__main__:Loading the interpolation points from file bivariate_study/output/LongitudinalMetricModel_interpolation_points.txt
INFO:__main__:The width for the metric interpolation is set to 0.3
INFO:__main__:I am setting the no_parallel_transport flag to False.
INFO:__main__:>> Reading 1-source initial modulation matrix from file: bivariate_study/output/final_modulation_matrix.txt
INFO:__main__:Setting initial sources from bivariate_study/output/LongitudinalMetricModel_sources.txt file
INFO:deformetrica.core.models.longitudinal_metric_learning:Acceleration factors max/min: (2.6998765, 133, 0.

608 good iterations out of 609


INFO:deformetrica.core.models.longitudinal_metric_learning:Tmin 45.92675018310547 Tmax 96.45048522949219 Update of the spatiotemporalframe: 2069 ms


655 good iterations out of 656


INFO:deformetrica.core.model_tools.manifolds.generic_geodesic:Want to estimate timepoints below tmin
INFO:deformetrica.core.model_tools.manifolds.generic_geodesic:Want to estimate timepoints above tmax
INFO:__main__:>> Initial noise variance set to 0.3232308194059048 based on the initial mean residual value.
INFO:deformetrica.core.models.longitudinal_metric_learning:>> The time shift variance prior degrees of freedom parameter is set to 255
INFO:deformetrica.core.models.longitudinal_metric_learning:>> The log-acceleration variance prior degrees of freedom parameter is set to the number of subjects: 255
INFO:deformetrica.core.models.longitudinal_metric_learning:('v0', False)
INFO:deformetrica.core.models.longitudinal_metric_learning:('p0', False)
INFO:deformetrica.core.models.longitudinal_metric_learning:('reference_time', False)
INFO:deformetrica.core.models.longitudinal_metric_learning:('onset_age_variance', False)
INFO:deformetrica.core.models.longitudinal_metric_learning:('log_accel

In [130]:
v0, p0, metric_parameters, modulation_matrix = model._fixed_effects_to_torch_tensors(False)
onset_ages, log_accelerations, sources = model._individual_RER_to_torch_tensors(averaged_rer, False)
t0 = model.get_reference_time()

absolute_times = model._compute_absolute_times(dataset.times, log_accelerations, onset_ages)

absolute_times_to_write = []
for elt in absolute_times:
    for e in elt.cpu().data.numpy():
        absolute_times_to_write.append(e)

#np.savetxt(os.path.join(Settings().output_dir, "LongitudinalMetricModel_absolute_times.txt"), np.array(absolute_times_to_write))

accelerations = torch.exp(log_accelerations)

model._update_spatiotemporal_reference_frame(absolute_times, p0, v0, metric_parameters,
                                            modulation_matrix)

INFO:deformetrica.core.models.longitudinal_metric_learning:Acceleration factors max/min: (2.7016509, 133, 0.32726613, 233)


610 good iterations out of 611


INFO:deformetrica.core.models.longitudinal_metric_learning:Tmin 45.84938049316406 Tmax 96.53067016601562 Update of the spatiotemporalframe: 2150 ms


657 good iterations out of 658


In [131]:
predictions = []
subject_ids = []
times = []

targets = dataset.deformable_objects

number_of_subjects = dataset.number_of_subjects
residuals = []

for i in range(number_of_subjects):
    predictions_i = []
    for j, t in enumerate(absolute_times[i]):
        if sources is not None:
            prediction = model.spatiotemporal_reference_frame.get_position(t, sources=sources[i])
        else:
            prediction = model.spatiotemporal_reference_frame.get_position(t)
        predictions_i.append(prediction.cpu().data.numpy())
        predictions.append(prediction.cpu().data.numpy())
        subject_ids.append(dataset.subject_ids[i])
        times.append(dataset.times[i][j])

    targets_i = targets[i].cpu().data.numpy()

    residuals.append(np.linalg.norm(predictions_i - targets_i, axis=0, ord=1)/len(absolute_times[i]))


INFO:deformetrica.core.model_tools.manifolds.generic_geodesic:Want to estimate timepoints below tmin
INFO:deformetrica.core.model_tools.manifolds.generic_geodesic:Want to estimate timepoints above tmax


In [132]:
predicted = data.copy()
predicted['adas_memory'] = [prediction[0] for prediction in predictions]
predicted['hippocampus'] = [prediction[1] for prediction in predictions]

In [133]:
diff = predicted - data
abs(diff).describe()

,adas_memory,hippocampus
count,942.000000,942.000000
mean,0.223971,0.413521
std,0.220601,0.231883
min,0.000156,0.003487
25%,0.058899,0.237536
50%,0.139755,0.407358
75%,0.306190,0.592808
max,0.854399,1.018662


In [134]:
predicted.describe()

,adas_memory,hippocampus
count,942.000000,942.000000
mean,0.374100,0.437007
std,0.285419,0.619094
min,-0.323792,-0.617507
25%,0.253271,-0.115440
50%,0.474453,0.482866
75%,0.556344,0.922737
max,0.937028,1.572575


In [129]:
data.describe(percentiles=[.9])

,adas_memory,hippocampus
count,942.000000,942.000000
mean,0.483881,0.430180
std,0.175045,0.181203
min,0.066667,0.000000
50%,0.488889,0.432856
90%,0.711111,0.648502
max,0.985111,1.000000
